- 1
- **create sqlite** database named **courses**
- 2
- using **python** + **peewee** :
    - 2.1
    - **create table Courses** (
        - name ,
        - price ,
        - category(relation) ,
        - content)
    - 2.2
    - create table Category (
        - name)
    - 2.3
    - **create dummy data**(Course) as json or csv (website)
    - 2.4
    - **insert** data from json or csv **into database**
    - 2.5
    - **insert** data from excel **into db**
    - 2.6
    - **get** all **courses**
    - 2.7
    - **get** how many courses in **each category**


---
---

- 1
- **create sqlite** database named **courses**

In [1]:
from peewee import(
                    SqliteDatabase,
                    Model,
                    CharField,
                    ForeignKeyField,
                    TextField,
                    IntegerField)

db = SqliteDatabase('courses.db')
print(f"{SqliteDatabase}")

<class 'peewee.SqliteDatabase'>


---

- 2
- using **python** + **peewee** :

---

---

- 2.2
    - create table Category (
        - name)

In [2]:
class Category(Model):
    name = CharField(unique=True)
    
    class Meta:
        database = db
print(f"{Category}")

<Model: Category>


---

- 2.1
    - **create table Courses** (
        - name ,
        - price ,
        - category(relation) ,
        - content)

In [3]:
class Course(Model):
    name = CharField()
    price = IntegerField()
    category = ForeignKeyField(Category, backref='courses')
    content = TextField()
    
    class Meta:
        database = db
print(f"{Course}")

<Model: Course>


---

In [4]:
db.connect()
db.create_tables([Category, Course])
print(f"{SqliteDatabase}{Category}{Category}")

<class 'peewee.SqliteDatabase'><Model: Category><Model: Category>


---

- 2.3
    - **create dummy data**(Course) as json or csv (website)
    

In [5]:
import csv
from faker import Faker

faker = Faker()

categories = [faker.word() for _ in range(5)] 
courses_data = []

for _ in range(100): 
    course_name = faker.sentence(nb_words=3)
    course_price = faker.random_int(min=100, max=1000)
    course_content = faker.text(max_nb_chars=200)
    category = faker.random_element(categories)
    courses_data.append({
        "name": course_name,
        "price": course_price,
        "category": category,
        "content": course_content
    })

with open('courses_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=["name", "price", "category", "content"])
    writer.writeheader()
    writer.writerows(courses_data)

print("Data exported to courses_data.csv successfully.")


Data exported to courses_data.csv successfully.


---

- 2.4
    - **insert** data from json or csv **into database**
    

In [6]:
import csv
from peewee import IntegrityError

csv_file_path = 'courses_data.csv'

with open(csv_file_path, newline='', encoding='utf-8') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
            category, created = Category.get_or_create(name=row['category'])
            
            Course.create(
                name=row['name'],
                price=float(row['price']),
                category=category,
                content=row['content']
            )
print(f"{csv_file_path}")

courses_data.csv


---

- 2.5
    - **insert** data from excel **into db**
    

In [7]:
from openpyxl import load_workbook
import os

excel_file_path = 'courses_data.xlsx'
workbook = load_workbook(filename=excel_file_path)
sheet = workbook.active  


for row in sheet.iter_rows(min_row=2, values_only=True):  
    name, price, category_name, content = row
    category, created = Category.get_or_create(name=category_name)
    Course.create(
        name=name,
        price=price,
        category=category,
        content=content
    )
print(f"{excel_file_path}")

courses_data.xlsx


---

- 2.6
    - **get** all **courses**
    

In [8]:
all_course = Course.select()
for cours in all_course:
    print([cours.name, cours.price, cours.category.name, cours.content])

['Shoulder success travel.', 140, 'wall', 'Young scientist into this. Foot score eat scene. Whatever upon before focus million today.']
['Expect will statement.', 791, 'list', 'Night right need structure go attention environment.\nBlue series seek. Not past camera in environmental quickly cultural. Detail step together inside sound because.']
['Per may.', 497, 'story', 'Represent young energy carry. Become rise protect value structure soon bank movie.\nHuge president campaign source suffer life that. Exist rich president save east respond.']
['Personal although common.', 793, 'happen', 'Experience will human soon against nice long. Great happen cause hit participant even. Less perhaps glass key music.\nSome example now agreement age. Easy safe wear station guess. Above TV popular.']
['Present town sign.', 185, 'number', 'Skin may politics factor hope.\nHundred picture fast fund class. Mother however forward power item rock save. Together late save bag realize among.']
['At task.', 902,

---

- 2.7
    - **get** how many courses in **each category**

In [9]:
from peewee import fn

category_counts = (Course
                   .select(Category.name, fn.COUNT(Course.id).alias('course_count'))
                   .join(Category)
                   .group_by(Category)
                   .order_by(Category.name))

for count in category_counts:
    print(count.category.name, count.course_count)


1 2
2 1
3 1
happen 22
list 16
number 15
story 30
wall 17
